# Coursera - Capstone - Week3 - Assignment - Tommy

## Requirements
REQ1: 
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood  
REQ2:
Only process the cells that have an assigned borough.  
REQ3: Ignore cells with a borough that is Not assigned.  
REQ4: More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.   
These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.   
REQ5: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making. 
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [1]:
#!pip install beautifulsoup4
#!pip install lxml
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge folium=0.5.0 --yes

In [2]:
#Scrape the List of Postal Codes of Canada
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import random
from geopy.geocoders import Nominatim

from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html

import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

# Get the data

In [3]:
#Scrape the List of Postal Codes of Canada
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

In [4]:
# Exploring the table column names
table_headers = []
for row in table.find_all('th'):
    table_headers.append(row.text.strip())
    #print(row.text.strip())
print(len(table_headers), "Columns")
print(table_headers)

3 Columns
['Postal Code', 'Borough', 'Neighborhood']


In [5]:
# The number of columns and the order is correct 
# it is only the naming for Postal Code that needs to change to PostalCode 
# before we start looping through the data we could have just preset the name of the columns
# ex. column_names = ['PostalCode','Borough','Neighborhood']
# we will instead read in the original column names and then later on using Pandas change the column name

In [6]:
#df = pd.DataFrame(columns = column_names)
df = pd.DataFrame(columns = table_headers)

# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

# Exploring the data

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [7]:
df.shape

(180, 3)

In [8]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# FIX for REQ1
dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [9]:
df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
# This is just removing the space in the column name

In [10]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### The Neighborhood column is not containing 'Not assigned' anymore -> instead it is empty

In [11]:
df.isnull().sum(axis=0)
#There is no null data

PostalCode      0
Borough         0
Neighborhood    0
dtype: int64

### double checking the nr and to make sure 'Not assigned' does not exist in other columns

In [12]:
d = (df == 'Not assigned').sum(axis=0)
d
# According to the instruction there has been 'Not assigned' in Neighborhood column but now only in Boroughs

PostalCode       0
Borough         77
Neighborhood     0
dtype: int64

# FIX for REQ2 & REQ3
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [13]:
# Dropping the rows where Borough is 'Not assigned'
removed_df = df[df['Borough']!='Not assigned'].reset_index(drop=True)
# remove 77 rows

In [14]:
removed_df.shape

(103, 3)

# FIX for REQ4
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.   
These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [15]:
# Combining the neighbourhoods with same Postalcode
df2 = removed_df.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)

# FIX for REQ5
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [16]:
# Replacing the name of the neighborhoods which are 'Not assigned' with names of Borough
df2['Neighborhood'] = np.where(df2['Neighborhood'] == 'Not assigned',df2['Borough'], df2['Neighborhood'])

## This seems to be an unnecessary step since their data does not have 'Not assigned' in  the Neighborhood column anymore ... only in Boroughs column

In [17]:
df2.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [18]:
df2.shape
# This is not lower in row number since their data on the website is already aggregated before getting the data - tutorial needs an update

(103, 3)

## Importing the csv file

In [19]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Merging the two Dataframes

In [20]:
lat_lon.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df3 = pd.merge(df2,lat_lon,on='PostalCode')
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Show only for Toronto

In [21]:
df4 = df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [23]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [26]:
k=5
toronto_clustering = df4.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df4.insert(0, 'Cluster Labels', kmeans.labels_)

In [27]:
df4

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,4,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,3,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,1,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [28]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighborhood'], df4['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [30]:
df4.shape

(39, 6)

In [ ]:
# End of Week 3 Assesment

# The 2 map images are available here:

https://github.com/tommyhagvallpulsen/Coursera_Capstone/blob/master/Week3/Coursera_Capstone_Week3_Map1.png
    
https://github.com/tommyhagvallpulsen/Coursera_Capstone/blob/master/Week3/Coursera_Capstone_Week3_Map1.png